In [1]:
%pip install tree-of-thoughts-llm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 34.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 39.4 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 51.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.5
    Uninstalling tqdm-4.66.5:
      Successfully uninstalled tqdm-4.66.5
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: multidict
    Found existing installation: multidict 6.1.0
    Uninstalling multidict-6.1.0:
      Successfully uninstal

In [4]:
import sys
import importlib

# Add your local repository path at the start of sys.path
local_repo_path = '/Users/henryyi/Documents/University/Post-Pretraining for LLM/library/tree-of-thought-llm'  # Replace with the actual path to 'tot' in your local repo
if local_repo_path not in sys.path:
    sys.path.insert(0, local_repo_path)

# Verify the path order
print("sys.path:", sys.path)

# Now import and reload the modules
import tot.methods.bfs
import tot.tasks.game24
importlib.reload(tot.methods.bfs)
importlib.reload(tot.tasks.game24)

# Import the specific functions/classes again
from tot.methods.bfs import solve, naive_solve
from tot.tasks.game24 import Game24Task

sys.path: ['/Users/henryyi/Documents/University/Post-Pretraining for LLM/library/tree-of-thought-llm', '/Users/henryyi/opt/anaconda3/envs/ML-Algorithm-LowerVersion/lib/python310.zip', '/Users/henryyi/opt/anaconda3/envs/ML-Algorithm-LowerVersion/lib/python3.10', '/Users/henryyi/opt/anaconda3/envs/ML-Algorithm-LowerVersion/lib/python3.10/lib-dynload', '', '/Users/henryyi/opt/anaconda3/envs/ML-Algorithm-LowerVersion/lib/python3.10/site-packages', '/Users/henryyi/opt/anaconda3/envs/ML-Algorithm-LowerVersion/lib/python3.10/site-packages/setuptools/_vendor']


In [2]:
import argparse
args = argparse.Namespace(backend='gpt-4', temperature=0.7, task='game24', naive_run=False, prompt_sample=None, method_generate='propose', method_evaluate='value', method_select='greedy', n_generate_sample=1, n_evaluate_sample=3, n_select_sample=5)

task = Game24Task()
ys, infos = solve(args, task, 900)
print(ys[0])

functools.partial(<function gpt at 0x10dcb9870>, model='gpt-4', temperature=0.7)
{'model': 'gpt-4', 'messages': [{'role': 'user', 'content': 'Input: 2 8 8 14\nPossible next steps:\n2 + 8 = 10 (left: 8 10 14)\n8 / 2 = 4 (left: 4 8 14)\n14 + 2 = 16 (left: 8 8 16)\n2 * 8 = 16 (left: 8 14 16)\n8 - 2 = 6 (left: 6 8 14)\n14 - 8 = 6 (left: 2 6 8)\n14 /  2 = 7 (left: 7 8 8)\n14 - 2 = 12 (left: 8 8 12)\nInput: 4 5 6 10\nPossible next steps:\n'}], 'temperature': 0.7, 'max_tokens': 1000, 'n': 1, 'stop': None}
{'model': 'gpt-4', 'messages': [{'role': 'user', 'content': 'Evaluate if given numbers can reach 24 (sure/likely/impossible)\n10 14\n10 + 14 = 24\nsure\n11 12\n11 + 12 = 23\n12 - 11 = 1\n11 * 12 = 132\n11 / 12 = 0.91\nimpossible\n4 4 10\n4 + 4 + 10 = 8 + 10 = 18\n4 * 10 - 4 = 40 - 4 = 36\n(10 - 4) * 4 = 6 * 4 = 24\nsure\n4 9 11\n9 + 11 + 4 = 20 + 4 = 24\nsure\n5 7 8\n5 + 7 + 8 = 12 + 8 = 20\n(8 - 5) * 7 = 3 * 7 = 21\nI cannot obtain 24 now, but numbers are within a reasonable range\nlikely\n

In [10]:
import argparse
import os
import json
from tot.tasks.text import TextTask

from tot.tasks import get_task
from tot.methods.bfs import solve, naive_solve
from tot.models import gpt_usage
args = argparse.Namespace(backend='gpt-4', 
                          task='text', 
                          task_start_index=0,
                          task_end_index=100,
                          naive_run=True,
                          prompt_sample='cot',
                          n_generate_sample=10,
                          temperature=1.0,
                          method_generate='sample',
                          method_evaluate='vote',
                          method_select='greedy',
                          )

task = TextTask()
logs, cnt_avg, cnt_any = [], 0, 0
if args.naive_run:
    file = f'./logs/{args.task}/{args.backend}_{args.temperature}_naive_{args.prompt_sample}_sample_{args.n_generate_sample}_start{args.task_start_index}_end{args.task_end_index}.json'
else:
    file = f'./logs/{args.task}/{args.backend}_{args.temperature}_{args.method_generate}{args.n_generate_sample}_{args.method_evaluate}{args.n_evaluate_sample}_{args.method_select}{args.n_select_sample}_start{args.task_start_index}_end{args.task_end_index}.json'
os.makedirs(os.path.dirname(file), exist_ok=True)


for i in range(args.task_start_index, args.task_end_index):
    # solve
    if args.naive_run:
        ys, info = naive_solve(args, task, i) 
    else:
        ys, info = solve(args, task, i)

    # log
    infos = [task.test_output(i, y) for y in ys]
    info.update({'idx': i, 'ys': ys, 'infos': infos, 'usage_so_far': gpt_usage(args.backend)})
    logs.append(info)
    with open(file, 'w') as f:
        json.dump(logs, f, indent=4)
    
    # log main metric
    accs = [info['r'] for info in infos]
    cnt_avg += sum(accs) / len(accs)
    cnt_any += any(accs)
    print(i, 'sum(accs)', sum(accs), 'cnt_avg', cnt_avg, 'cnt_any', cnt_any, '\n')

n = args.task_end_index - args.task_start_index
print(cnt_avg / n, cnt_any / n)
print('usage_so_far', gpt_usage(args.backend))


functools.partial(<function gpt at 0x10dcb9870>, model='gpt-4', temperature=1.0)
{'model': 'gpt-4', 'messages': [{'role': 'user', 'content': "\nWrite a coherent passage of 4 short paragraphs. The end sentence of each paragraph must be: It isn't difficult to do a handstand if you just stand on your hands. It caught him off guard that space smelled of seared steak. When she didn’t like a guy who was trying to pick her up, she started using sign language. Each person who knows you has a different perception of who you are.\n\n\nMake a plan then write. Your output should be of the following format:\n\nPlan:\nYour plan here.\n\nPassage:\nYour passage here.\n"}], 'temperature': 1.0, 'max_tokens': 1000, 'n': 10, 'stop': None}
{'model': 'gpt-4', 'messages': [{'role': 'user', 'content': "\nWrite a coherent passage of 4 short paragraphs. The end sentence of each paragraph must be: It isn't difficult to do a handstand if you just stand on your hands. It caught him off guard that space smelled of 

KeyboardInterrupt: 

In [3]:
print(ys[0])

10 - 4 = 6 (left: 5 6 6)
5 * 6 = 30 (left: 6 30)
30 - 6 = 24 (left: 24)
Answer: (5 * (10 - 4)) - 6 = 24

